In [ ]:
import os
print(f"当前目录: {os.getcwd()}")
if 'arch-gaussian' in os.listdir():
    JUPYTER_ROOT = os.getcwd()
    os.chdir('arch-gaussian')
    print(f"更改后的目录: {os.getcwd()}")

In [ ]:
import config
# 基础设置
config.scene_name = "TestCity7" # 输入文件夹名称
config.output_name = ""  # 输出文件夹名称
config.sh_degree = 3  # 0~3， 改为0可以缩小文件大小，但是无法被现有的unity工具识别， 推荐使用默认3
config.epochs = 0  # 训练轮次， 3000， 7000， 15000， 30000
config.loaded_iter = 3000 # 加载训练好的模型
# 高级设置
config.resolution = 1 # 是否压缩图像，1为原尺寸，-1为自动压缩到不大于1600像素， 支持2的次方的数字
config.densify_until_iter = 15000  # 致密化结束的轮次
config.densify_grad_threshold = 0.0002  # 致密化的阈值，越小增加的越快

# 配置结束，更新配置文件
config.update_colmap_args()
config.update_args()

# 打印修改过后的参数
config.print_updated_colmap_args()
config.print_updated_args()


In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("./src")
import os
import torch
import numpy as np
np.set_printoptions(suppress=True)
from config import args  # 正式导入args

In [ ]:
# 创建scene info
from manager.scene_manager import load_and_fix_scene

scene_info = load_and_fix_scene(args)

In [ ]:
from manager.camera_manager import CameraManager
cm = CameraManager()
train_cameras, test_cameras = cm.create_cameras(args, scene_info)


In [ ]:
from manager.gaussian_manager import create_gaussian_from_ply, create_gaussian_from_scene_info
if args.loaded_iter:
    print("Creating gaussians from ply")
    gaussians = create_gaussian_from_ply(args.sh_degree, os.path.join(args.model_path,"point_cloud", "iteration_" + str(args.loaded_iter),"point_cloud.ply"))
else:
    print("Creating gaussians from scene info")
    gaussians = create_gaussian_from_scene_info(args.sh_degree, scene_info)

print(f"num points: {gaussians.get_xyz.shape[0]}")

## 使用封装好的Gaussian Manager管理gaussian

In [ ]:
from manager.gaussian_manager import GaussianManager
gm = GaussianManager(args, gaussians) # 从gaussians创建GaussianManager
cam = cm.pick_camera(0)
image = gm.render(cam, convert_to_pil=True)
image

In [ ]:
from manager.display_manager import Box, Geometry, Drawer
bbox = (np.array([-0.5,0,-2]), np.array([0.5,0.5,-1.9]))

mask = gm.position_mask(*bbox)

box = Box(*bbox)
box.line_width = 29
drawer = Drawer()
drawer.add_geometry(box)

with gm.virtual():

    gm.paint_by_mask(mask)
    image = gm.render(cam, convert_to_pil= True)
    image = drawer.draw(cam, image,overwrite=True)
image